# Audiobooks business case

The targets are a Boolean variable 0, or 1. We are taking a period of 2 years in our inputs, and the next 6 months as 
targets. So, in fact, we are predicting if: based on the last 2 years of activity and engagement, a customer will convert 
in the next 6 months. 6 months sounds like a reasonable time. If they don't convert after 6 months, chances are they've
gone to a competitor or didn't like the Audiobook way of digesting information..

The task is to create a machine learning algorithm which is able to predict if a customer will buy again.

This is a classification problem with two classes: won't buy and will buy, represented by Os and 1s.

## Import relevant Lib and extract the data from .csv

In [1]:
import numpy as np
from sklearn import preprocessing


raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

## Shuffle the unscaled data

In [2]:
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

## Balance the dataSet

In [3]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

## Standardize the inputs

In [4]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

## Shuffle the data

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

## Split the dataset into train, validation, and test

In [6]:
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1789.0 3579 0.49986029617211514
225.0 447 0.5033557046979866
223.0 448 0.49776785714285715


## Save the three datasets in *.npz

In [7]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

# Create the machine learning algorithm¶

## Import the relevant libraries

In [8]:
import numpy as np
import tensorflow as tf

## Load data

In [9]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

C:\Users\yeeme\AppData\Local\Temp/ipykernel_10104/3894879715.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['inputs'].astype(np.float)
C:\Users\yeeme\AppData\Local\Temp/ipykernel_10104/3894879715.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org

## Model
Outline, optimizers, loss, early stopping and training

In [10]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([   
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_inputs, 
          train_targets, 
          batch_size=batch_size, 
          epochs=max_epochs, 
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets), 
          verbose = 2 
          )  

Epoch 1/100
36/36 - 1s - loss: 0.5803 - accuracy: 0.6857 - val_loss: 0.5019 - val_accuracy: 0.7942 - 1s/epoch - 37ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4772 - accuracy: 0.7611 - val_loss: 0.4269 - val_accuracy: 0.7919 - 104ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4287 - accuracy: 0.7795 - val_loss: 0.3878 - val_accuracy: 0.8076 - 77ms/epoch - 2ms/step
Epoch 4/100
36/36 - 0s - loss: 0.4027 - accuracy: 0.7930 - val_loss: 0.3708 - val_accuracy: 0.8210 - 68ms/epoch - 2ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3890 - accuracy: 0.7994 - val_loss: 0.3576 - val_accuracy: 0.8255 - 73ms/epoch - 2ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3842 - accuracy: 0.7960 - val_loss: 0.3481 - val_accuracy: 0.8322 - 67ms/epoch - 2ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3769 - accuracy: 0.7994 - val_loss: 0.3472 - val_accuracy: 0.8300 - 74ms/epoch - 2ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3717 - accuracy: 0.8058 - val_loss: 0.3431 - val_accuracy: 0.8255 - 74ms/epoch - 2ms/step
Epoch 9/100
36/3

## Test the model

In [11]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 3ms/step - loss: 0.3198 - accuracy: 0.8415


In [12]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.32. Test accuracy: 84.15%
